# Генеративные модели для табличных временных рядов II

Цели:

* познакомиться с современными моделями
* рассмотреть диффузионные модели, модели на основе LLM и дифференциальных уравнений


Содержание:

- Diffusion Generative Models (DGM)
  - Модели для генерации финансовых временных рядов
  - Задача генерации с ограничениями
- LLM
- Misc
  - Chronos
  - Дифференциальные уравнения

## Diffusion Generative Models (DGM)

### Модели для генерации финансовых временных рядов

#### FinDiff

[FinDiff: Diffusion Models for Financial Tabular Data Generation, 2023](https://arxiv.org/pdf/2309.01472v1)

FinDiff - дифузионная модель для финансовых табличных данных смешанной модальности, включающих числовые и категориальные атрибуты.

Числовые атрибуты  нормализуются, а категориальные атрибуты отображаются в пространство эмбеддингов (embedding encoding) и конкатенируются с числовыми. Затем комбинируются с синусоидальными позиционными эмбеддингами времени и целевого атрибута. При генерации выбирается категориальное значение с ближайшим эмбеддингом, а числовые атрибуты денормализуются.

<figure>
<img src="https://raw.githubusercontent.com/sswt/dive2gai/w14/.github/images/w14/findiff.png" alt="FinDiff arch" width="100%">
<figcaption align = "center"> Image credit: FinDiff paper</figcaption>
</figure>

Победила TVAE, CTGAN и TabDDPM на трёх датасетах в режиме TSTR, но датасеты дневные или месячные агрегаты.

Отдельно хочется отметить, что абляционные тесты показали преимущество квантильного трансформера над стандартным шкалированием и степенной трансформацией Йео-Джонсона.

В чём отличие от других - TabDDPM кодирует категориальные атрибуты мультиномиальными диффузионными моделями, которые используют one-hot кодирование, что плохо масштабируется на большую кардинальность.

Здесь, чтобы обойти дискретную природу данных, токены проецируются в непрерывное пространство эмбеддингов. Используется [Self-conditioned Embedding Diffusion (SED)](https://arxiv.org/pdf/2211.04236) — это непрерывный механизм диффузии, который работает с эмбеддингами токенов и позволяет обучать гибкие и масштабируемые диффузионные модели как для условной, так и для безусловной генерации текста.

#### FedTabDiff

[FedTabDiff: Federated Learning of Diffusion Probabilistic Models for Synthetic Mixed-Type Tabular Data Generation, 2024](https://arxiv.org/pdf/2401.06263)

Дальнейшее развитие FinDiff - модель федеративной табличной диффузии (FedTabDiff), предназначенная для создания смешанных табличных данных без централизованного доступа к исходным наборам данных.

Недавно была предложена концепция Федеративного Обучения (FL) (McMahan et al. 2017; McMahan and Ramage 2017), в которой несколько устройств, таких как смартфоны или серверы, совместно обучают модели искусственного интеллекта под руководством центрального сервера (Kairouz et al. 2019). Обучающие данные остаются децентрализованными, обеспечивая путь к обучению общей модели без прямого обмена данными. Существует несколько моделей для обучения Федеративной Диффузии.

Каждый клиент $ \omega_i $ поддерживает свою версию модели FinDiff и вносит свой вклад в обучение центральной модели. Федеративная оптимизация использует синхронную схему обновления на протяжении $ r = 1, \dots, R $ раундов коммуникации. В каждом раунде выбирается подмножество клиентов. Эти клиенты получают текущие параметры центральной модели, выполняют оптимизацию, а затем отправляют обновленные параметры модели для агрегации. Датасеты на клиентах не являются случайной независимой выборкой. Вместе они формируют лучшую модель, чем если бы обучение проводилось по отдельности.

<figure>
<img src="https://raw.githubusercontent.com/sswt/dive2gai/w14/.github/images/w14/fedtabdiff.png" alt="Federated learning" width="75%">
<figcaption align = "center"> Image credit: FedTabDiff paper</figcaption>
</figure>

### Задача генерации с ограничениями

[On the Constrained Time-Series Generation Problem, 2023](https://proceedings.neurips.cc/paper_files/paper/2023/file/bfb6a69c0d9e2bc596e1cd31f16fcdde-Paper-Conference.pdf)

В работе рассматриваются вещественные многомерные временные ряды.

Часто при генерации временных рядов нужно удовлетворять численным ограничениям и есть требованиям по схожести с т.з. распределения (назовём это "реализм").

В работе предложен набор методов для задачи генерации временных рядов с ограничениями и обеспечению эффективного сэмплирования (без переобучения модели) сохраняя реализм генерируемых данных.

В работе есть короткий обзор источников с известными методами введения ограничений при генерации:

- Дополнительные ограничения обычно вводятся путём штрафов пропорционально объёму генерируемых невалидных данных или добавлением регуляризации в лосс.
- Альтернативные подходы кодируют ограничения и подают на вход модели, как условие или отбрасывают и заново генерируют ряд, если он не удовлетворяет ограничениям.

В данной работе в первую очередь задачу сводят к задаче оптимизации с ограничениями (Constrained Optimization Problem) и генерируют данные исходя из явной формулировки свойств данных и ограничений.

Затем, чтобы избежать необходимости явного задания свойств данных используется диффузионная модель, которая неявно выучивает свойства данных из обучающей выборки. В частности, вводится DiffTime, подход на основе условных диффузионнных моделей (conditional denoising diffusion models), с условиями в виде ограничений.

Дополнительно, показывается, что любой тип ограничений может быть применён к диффузионной модели, штрафуя пропорционально нарушениям ограничений при генерации. Такой подход, называющийся Loss-DiffTime, показывает хорошую производительность с эффективным инференсом, но требует переобучения при изменении ограничений.

И наконец 🎉, предлагается управляемая диффузионная модель **Guided-DiffTime**, которая не требует повторного обучения при изменении ограничений — при инференсе она может корректировать процесс генерации временных рядов на основе входных ограничений.

То есть основные результаты по мнению авторов:

- Формальное определили задачи временных рядов с ограничениями и различных типов ограничений для синтетических временных рядов.
- Предложили и сравнили различные подходы к генерации синтетических временных рядов.
- Показали, как COP также может быть использован для настройки временных рядов постфактум, так что синтетические временные ряды, созданные любой DGM, могут быть скорректированы для гарантии соблюдения ограничений.
- DiffTime превосходит существующие передовые модели в генерации временных рядов без ограничений, в то время как Guided-DiffTime является единственным решением, где повторное обучение для новых ограничений не требуется.

#### Ограничения

<figure>
<img src="https://raw.githubusercontent.com/sswt/dive2gai/w14/.github/images/w14/ts_constraints.png" alt="TS constraints" width="75%">
<figcaption align = "center"> Image credit: GuidedDiffTime paper</figcaption>
</figure>

Ограничения — как те, что показаны на рисунке выше — определяются в виде кортежей $⟨t, f⟩ ∈ C$, где $t$ (тип ограничения) может быть **мягким или жестким**, а $f$ является дифференцируемой функцией.

Если тип жесткий, то $f$ может быть неравенством $f(\hat{x}) \leq 0$ или равенством $f(\hat{x}) = 0$, где $\hat{x}$ — это сгенерированный синтетический временной ряд. Жесткие ограничения должны соблюдаться в сгенерированном временном ряде, иначе ряд отбрасывается.

Мягкие ограничения имеют форму $ f: \chi \to \mathbb{R} $, значение которых мы хотим оптимизировать (минимизировать). Поэтому, по определению, мягкие ограничения не требуют отбраковки выборок.

Ограничения могут быть определены **относительно отдельных временных рядов** $\hat{x}$ **или на уровне всего набора данных** (ограничения на распределение).

В качестве типа мягкого ограничения, тренд-линии (см. Figure 1.b) определяются как временной ряд $s \in \chi$. Это ограничение указывает генеративному методу минимизировать $L_2$ расстояние между трендом и соответствующими точками синтетического временного ряда. Формально, синтетический временной ряд $\hat{x}$ будет оптимизирован для минимизации $\|s - \hat{x}\|_2^2$.

Кроме того, как мягкие, так и жесткие ограничения могут быть классифицированы **на локальные и глобальные ограничения**.

Глобальные ограничения — это те, которые сравниваются по всем точкам временного ряда. Например, мы можем установить, что $x_{i,j} \leq x_{3,0}$ для всех $(i, j) \in [L] \times [K]$ таким образом, что максимальное значение находится в $x_{3,0}$.

Локальные ограничения относятся только к подмножеству точек. Например, требование $x_{i,j} = 2.5$ для данной точки $(i, j) \in [L] \times [K]$. Мы называем этот вид ограничений фиксированными точками (см. Figure 1.c), так как они требуют, чтобы значение временного ряда в этой точке было зафиксировано на определенном уровне. Множество всех фиксированных точек обозначается как $R$, где каждый элемент $r_{i,j} \in R$ и $(i, j) \in [L] \times [K]$.

Вышеупомянутые типы ограничений являются явными. Кроме того, задача генерации данных требует статистического сходства между входными и синтетическими наборами данных, что может быть либо встроено в метод генерации (например, по замыслу GAN генерируют данные, которые распределительно похожи на входные), либо явно указано как ограничения в модели (например, автокорреляционное сходство может быть явным ограничением).

Методы, представленные здесь, предполагают, что ограничения дифференцируемы. Это необходимо для диффузионных моделей, где ограничения нужно учитывать в процессе обучения или вывода. Если функции дифференцируемы, то простой способ включения их в функцию потерь выглядит так:

$$
\text{loss}(x, \hat{x}) = \text{objective_loss}(x, \hat{x}) + \lambda_g \text{ReLU}(g(\hat{x})) + \lambda_h h(\hat{x})^2,
$$

где $g(\hat{x})$ и $h(\hat{x})$ являются соответственно ограничениями в форме неравенства и равенства, которые добавляются как мягкие ограничения в функцию потерь с штрафными коэффициентами $\lambda_g$ и $\lambda_h$. Однако включение ограничений в функцию потерь не гарантирует результа, удовлетворяющий ограничениям, но обеспечивает хорошие кандидаты или начальные решения, которые мы затем можно доработать (т.е., настроить для гарантии выполнения ограничений). Если ограничения не дифференцируемы, можно использовать подходы, которые вычисляют потери для такого "правила" с использованием перестановок, как в работе *Controlling neural networks with rule representations*.


## LLM

### UniTS

[Project page](https://zitniklab.hms.harvard.edu/projects/UniTS/), [paper](https://arxiv.org/pdf/2403.00131) and [code](https://github.com/mims-harvard/UniTS)

С развитием Self-Supervised Learning, базовые модели (foundation models), особенно LLM, изменили многое. Вместо того чтобы обучать множество моделей, специфичных для определенных задач, мы можем адаптировать одну предварительно обученную модель к множеству задач с помощью few-shot промптинга или файнтюнинга. Однако текущие базовые модели применяются к последовательностям, но не к временным рядам. Авторы предложили UniTS, унифицированную модель временных рядов, которая поддерживает универсальную спецификацию задач, объединяющую классификацию, прогнозирование, импутацию и обнаружение аномалий.

Это достигается благодаря новой унифицированной backbone-сети, которая включает в себя attention для последовательностей и переменных вместе с динамическим линейным оператором и обучается как единая модель.

UniTS обладает замечательными возможностями zero-shot, few-shot и обучения промптами для новых областей данных и задач.

Временной ряд нарезается на неперекрывающиеся патчи, которые потом пропускаются через обучаемый слой линейной проекции (эмбеддинг) и к ним прибавляется обучаемый position encoding вектор. Модель работает с многомерными вещественными рядами, но каждый ряд рассматривается по отдельности.

Модель состоит из N блоков, которые называются UNITS блоками и имеют следующие компоненты:

- Sequence Multi-Head Self-Attention (MHSA)
- Variable MHSA
- Dynamic MLP
- модули шлюзов

Для Variable MSHA, чтобы выучить глобальные связи между переменными по всем точкам последовательности, минимизируя при этом вычислительные затраты на длинные последовательности, усредняются $Q$ и $K$ по длине последовательности, чтобы получить общие $\hat Q$ and $\hat K$.

В Dynamic MLP на рисунке видим DyLinear - для учёта временных рядов переменной длительности.

Шлюзы, как в LSTM - поэлементное умножение входа на его линейная проекцию с сигмоидой на выходе.

<figure>
<img src="https://raw.githubusercontent.com/sswt/dive2gai/w14/.github/images/w14/units.png" alt="Unified TS model" width="75%">
<figcaption align = "center"> Image credit: UniTS paper</figcaption>
</figure>


[Evaluating Large Language Models on Time Series Feature Understanding:
A Comprehensive Taxonomy and Benchmark, 2024](https://arxiv.org/pdf/2404.16563v1)

Анализ преимуществ и ограничений современных LLM в понимании временных рядов, показывая, какие особенности эти модели усваивают, а какие нет. Кроме того, ислледована чувствительность LLM к таким факторам, как форматирование данных, позиция запрашиваемых точек внутри ряда и общая длина временного ряда.

Вводится таксономия временных рядов

<figure>
<img src="https://raw.githubusercontent.com/sswt/dive2gai/w14/.github/images/w14/ts_taxonomy.png" alt="TS taxonomy" width="75%">
<figcaption align = "center"> Image credit: This paper</figcaption>
</figure>

Генерируются синтетические временные ряды и исследуются возможности по обнаружению и локализацию вышеприведённых признаков.

Для обнаружения например, подаётся промпт - "Есть ли тут тренд?" и временной ряд прямо текстом. При этом пробовали разное форматирование текста, от CSV, до обогащённого описания. Для классификации промпт выглядел так `"Select one of the following answers: (a) the time series has a positive trend, (b) the time series has a negative trend. Provide your answer as either (a) or (b)."`.

Исследовались GPT4, GPT3.5, Llama2-13B и Vicuna-13B. Огромное количество метрик и вы знаете, кто победил.

[Foundation Models for Time Series Analysis: A Tutorial and Survey, 2024](https://arxiv.org/pdf/2403.14735)

Детальный анализ базовых моделей для временных рядов с попыткой понять внутренние механизмы, которые объясняют, почему и как базовые модели полезны для анализа временных рядов.

В числе типов рассматриваются наши любимые Event Sequences, как временно упорядоченное множество событий. Но информации очень мало.

Этот абзац здесь наверно только из-за ссылки на работу `Where Would I Go Next? Large Language Models as Human Mobility Predictors, 2024`, где упоминается [NYC and Tokyo Check-in Dataset](https://sites.google.com/site/yangdingqi/home/foursquare-dataset), который может быть нам полезен, так как там есть пользователь, место, время посещения и категория места. Там же есть большоий Global-scale Check-in Dataset, содержещий 33М событий от 266К пользователей.






## Misc

### Chronos

[Chronos: Learning the Language of Time Series](https://arxiv.org/pdf/2403.07815)

Большая работа от amazon, где обучают единую модель на архитектуре T5 на большое количество датасетов для предсказания:

- значительно превосходит другие методы на наборах данных, которые были частью обучающей выборки
- имеет сопоставимую и иногда превосходящую производительность в режиме zero-shot на новых наборах данных по сравнению с методами, которые были специально на них обучены

Сейчас есть много вариантов по использованию предобученных LLM для прогноза, но использовать GPT и LLama для прогноза временных рядов - расстраивать Грету Тунберг. Также известны модели поменьше, но их приходится дообучать или файнтюнить под задачу. Chronos тренировали from scratch.

Chronos выполняет токенизацию временных рядов в дискретные бины путем простого масштабирования (mean scaling) и квантизации вещественных значений на B бинов. Таким образом, мы можем обучать стандартные языковые модели на этом "языке временных рядов" без изменений в архитектуре модели. Удивительно, но этот простой подход оказывается эффективным и действенным, подчеркивая потенциал архитектур языковых моделей для решения широкого спектра задач временных рядов с минимальными модификациями.

<figure>
<img src="https://raw.githubusercontent.com/sswt/dive2gai/w14/.github/images/w14/chronos.png" alt="Chronos" width="75%">
<figcaption align = "center"> Image credit: Chronos paper</figcaption>
</figure>

#### Как обучали

Для обычных LLM есть огромное количество данных, а для временных рядов, даже когда собрали обширную коллекцию публичных наборов данных, их было недостаточно.

Авторы применили очень интересные методы аугментации, включая TSMixup и KernelSynth. TSMixup случайным образом выбирает множество временных рядов из всего набора обучающих данных и генерирует новые временные ряды на основе их выпуклой комбинации. KernelSynth использует гауссовские процессы для генерации синтетических временных рядов путем случайного составления ядерных функций.

Всего смогли собрали 28 наборов данных для обучения моделей Chronos, состоящих примерно из 890 тысяч одномерных временных рядов с общим количеством около 84 миллиардов наблюдений (токенов). Ряды очень разные и из разных областей, брали даже с датасетов Kaggle. С помощью аугментаций датасет расширили до 10M на основе TSMixup и добавили 1M синтетческих рядов, сгенерированных с помощью гауссовских процессов.

<figure>
<img src="https://raw.githubusercontent.com/sswt/dive2gai/w14/.github/images/w14/chronos_data.png" alt="Chronos datasets" width="75%">
<figcaption align = "center"> Image credit: Chronos paper</figcaption>
</figure>

Проводили очень много экспериментов и результаты очень впечатляющие.


Обучали разные размеры T5 (Mini (20M), Small (46M), Base (200M) and Large (710M)), Small обучался 8 часов на 8xA100 (40 Gb), в течении 8 часов, что стоило 250$.

По скорости инференса работает довольно шустро Small - 15 мс, можно даже в MARU использовать.


### Дифференциальные уравнения

Управляемые нейросетью дифференциальные уравнения (ДУ) как непрерывный аналог RNN.

RNN - популярный выбор модели для последовательных данных, таких как временные ряды. Сами данные часто предполагается быть последовательностью наблюдений из какого-то процесса, и RNN могут быть интерпретированы как дискретное приближение к некоторой функции этого процесса. Однако такая дискретизация обычно нарушается, если последовательность нерегулярна во времени.

Более элегантным подходом является осознание того, что поскольку основной процесс развивается в непрерывном времени, наши модели также должны быть непрерывными. Например, [6, 7, 8, 9] включают экспоненциальное затухание между наблюдениями, [10, 11] гибридизируют гауссовский процесс с традиционными моделями нейронных сетей, [12] приближают основной непрерывно-временной процесс, а [13, 14] адаптируют рекуррентные нейронные сети, позволяя некоторому скрытому состоянию эволюционировать как обыкновенному дифференциальному уравнению (ODE). Именно этот последний вариант рассматривается в работе `NCDE for Irregular Time Series`.

![](https://github.com/patrick-kidger/NeuralCDE/blob/master/imgs/main.png?raw=true)

- [Neural Controlled Differential Equations for Irregular Time Series, 2020](https://arxiv.org/pdf/2005.08926) - ОДУ для нерегулярных временных рядов
- [Neural Rough Differential Equations for Long Time Series, 2021](https://arxiv.org/pdf/2009.08295) - использование "грубых" дифференциальных уравнений
- [GT-GAN: General Purpose Time Series Synthesis with Generative Adversarial Networks, 2022](https://arxiv.org/pdf/2210.02040) - модель "общего назначения" для регулярных и нерегулярных временных рядов, для этого включает автоэнкодер и генератор работает в его скрытом пространстве. Комбайн NCDE и нормализующих потоков. Также замечен GRU-Δt, который принимает временную длительность между точками на вход.
- [Generative Modeling of Regular and Irregular Time Series Data via Koopman VAEs, 2023](https://arxiv.org/pdf/2310.02619) - использовали Коопмановскую теорию для динамических систем (что бы это не значило). Побил всех, включая GT-GAN.